# 15. 딕셔너리 삽입 순서에 의존할 떄는 조심하라

파이썬 3.5 이전에는 딕셔너리에 대해 이터레이션을 수행하면 키를 임의의 순서로 돌려줬으며, 이터레이션 순서는 원소가 삽입된 순서와 일치하지 않았다.

In [1]:
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy',
}
# 파이써 3.5에서는 {'dog': 'puppy', 'cat': 'kitten'}가 출력되지만 3.7 이후에서는 {'cat': 'kitten', 'dog': 'puppy'}
print(baby_names)

{'cat': 'kitten', 'dog': 'puppy'}


이러한 이유는 예전 딕셔너리 구현이 내장 hash 함수와 파이썬 인터프리터가 시작할 때 초기화되는 난수 씨드를 사용하는 해시 테이블 알고리즘으로 만들어졌기 때문이다.

파이썬 3.6부터 삽입순서를 보존하도록 개선되었다. (3.7 아닌가??) -> 3.7부터 파이썬 언어 명세에 내용이 포함됨

In [3]:
print(list(baby_names.keys()))
print(list(baby_names.values()))
print(list(baby_names.items()))
print(baby_names.popitem())

['cat']
['kitten']
[('cat', 'kitten')]
('cat', 'kitten')


함수에 대한 키워드 인자는 예전에는 이러한 이슈때문에 뒤죽박죽 처럼 보였다.

In [4]:
def my_func(**kwargs):
    for key, value in kwargs.items():
        print(f'{key} = {value}')

In [6]:
my_func(goose='gosling', kangaroo='joey')

goose = gosling
kangaroo = joey


In [7]:
class MyClass:
    def __init__(self):
        self.alligator = 'hatchling'
        self.elephant = 'calf'

In [8]:
a = MyClass()
for key, value in a.__dict__.items():
    print(f'{key} = {value}')

alligator = hatchling
elephant = calf


collections에 포함되어 있는 OrderDict은 dict과는 특성이 조금 다름.

popitem 호출을 많이 한다면, OrderDict이 더 나을 수 있다.

파이썬에서는 list, dict 등의 프로토콜을 흉내내는 커스텀 컨테이너 타입을 정의할 수 있다.

대부분의 경우 엄격한 클래스 계층보다 객체의 동작이 객체의 실질적인 타입을 결정하는 **덕 타이핑**에 의존하며, 이로인해 함정에 빠질 수 있다.

In [9]:
votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 863,
}

In [10]:
def populate_ranks(votes, ranks):
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

In [11]:
def get_winner(ranks):
    return next(iter(ranks))

In [12]:
ranks = {}
populate_ranks(votes, ranks)
print(ranks)
winner = get_winner(ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
otter


만약 등수가 아닌 알파벳 순으로 표시해야 할때, collections.abc 모듈을 사용해 딕셔너리와 비슷하지만 알파벳 순서대로 이터레이션 해주는 클래스를 새로 정의할 수 있다.

In [13]:
from collections.abc import MutableMapping

In [14]:
class SortedDict(MutableMapping):
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self, key):
        del self.data[key]

    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key

    def __len__(self):
        return len(self.data)

In [15]:
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)


{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


fox 가 출력되므로 요구사항에 맞지 않다.

해결 방법

1. ranks 딕셔너리가 어떤 특정 순서로 이터레이션된다고 가정하지 않고 get_winner 함수를 구현하는 것

In [16]:
def get_winner(ranks):
    for name, rank in ranks.items():
        if rank == 1:
            return name

In [17]:
winner = get_winner(sorted_ranks)
print(winner)

otter


2. 함수 맨 앞에 ranks의 타입이 우리가 원하는 타입인지 검사하는 코드를 추가

In [18]:
def get_winner(ranks):
    if not isinstance(ranks, dict):
        raise TypeError('dict 인스턴스가 필요합니다')
    return next(iter(ranks))

In [19]:
get_winner(sorted_ranks)

TypeError: dict 인스턴스가 필요합니다

3. 타입 annotation을 사용해서 get_winner에 전달 되는 값이 딕셔너리와 비슷한 동작을 하는 MutableMapping이 아니라 dict 인스턴스가 되도록 강제하는 것

In [20]:
from typing import Dict, MutableMapping

In [21]:
def populate_ranks(votes: Dict[str, int],
                   ranks: Dict[str, int]) -> None:
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

In [22]:
def get_winner(ranks: Dict[str, int]) -> str:
    return next(iter(ranks))

In [23]:
class SortedDict(MutableMapping[str, int]):
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self, key):
        del self.data[key]

    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key

    def __len__(self):
        return len(self.data)

In [24]:
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


In [27]:
!pip install mypy

Looking in indexes: http://mirror.kakao.com/pypi/simple
     |████████████████████████████████| 22.0 MB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 768 kB 15.7 MB/s eta 0:00:01


In [28]:
!python3 -m mypy --strict dict_test.py

dict_test.py:6: error: Argument "key" to "sort" of "list" has incompatible type overloaded function; expected "Callable[[str], _SupportsLessThan]"
dict_test.py:16: error: Function is missing a return type annotation
dict_test.py:16: note: Use "-> None" if function does not return a value
dict_test.py:19: error: Function is missing a type annotation
dict_test.py:22: error: Function is missing a type annotation
dict_test.py:25: error: Function is missing a type annotation
dict_test.py:28: error: Function is missing a return type annotation
dict_test.py:34: error: Function is missing a return type annotation
dict_test.py:43: error: Call to untyped function "SortedDict" in typed context
dict_test.py:44: error: Argument 2 to "populate_ranks" has incompatible type "SortedDict"; expected "Dict[str, int]"
dict_test.py:46: error: Argument 1 to "get_winner" has incompatible type "SortedDict"; expected "Dict[str, int]"
Found 10 errors in 1 file (checked 1 source file)


## 기억해야 할 내용
- 파이썬 3.7부터는 dict 인스턴스에 들어 있는 내용을 이터레이션할 때 키를 삽입한 순서대로 돌려받는다는 사실에 의존할 수 있다.
- 파이썬은 dict는 아니지만 딕셔너리와 비슷한 객체를 쉽게 만들 수 있게 해준다. 이런 타입의 경우 키 삽입 순서가 그대로 보존된다고 가정할 수 없다.
- 딕셔너리와 비슷한 클래스를 조심스럽게 다루는 방법으로 dict 인스턴스의 삽입 순서 보존에 의존하지 않고 코드를 작성하는 방법, 실행 시점에 명시적으로 dict 타입을 검사하는 방법, 타입 애너테이션과 정적분석을 사용해 dict 값을 요구하는 방법이 있다.